## Naive Bayes

In this notebook we are are going to implement the Naive Bayes from scratch to understand the application of Bayes Theorem by creating a Text Classification model.

#### Bayes Theorem

P(Class | Document) = (P(Document | Class) * P(Class))  / P(Document)

P(Class | Document) = Probability that the document belong to a particular class given its content

P(Document | Class) = The Likelihood of the document's content given that it belong to the class. This is where the "naive" assumption comes in; we assume that the presence of one word in a document is independent of the presence of the other word.

P(Class) = The prior probability of the class(how often this class occur in the dataset)

P(Document) = The probability of the document

Prior Probability: P(Class) Calculated from the frequency of each class in the training set.

Likelihood: P(Document | Class) Assuming Independence. This will become the product of all the probabilities of P(Wi | Class)

Where Wi is each word in a document.

P(Wi | Class) = (Count(Wi, Class) + alpha)/(Count(AllWords, class) + alpha * |number of uniqe words|)

Prediction: Calculate the probability for each class using the log-sum trick to avoid underflow, since we are multiplying many probabilities.

In [55]:
import numpy as np
from collections import defaultdict
import re

class NaiveBayes:
    def __init__(self, alpha=1.0):
        # Alpha is for Laplace smoothing to handle words not seen in training
        self.alpha = alpha
        self.classes = None
        self.vocab = None
        self.class_word_counts = defaultdict(lambda: defaultdict(int))
        self.class_counts = defaultdict(int)

    def tokenize(self, text):
        # Convert to lowercase and split text into words
        return re.findall(r'\w+', text.lower())

    def fit(self, X, y):
        # Initialize classes and vocabulary
        self.classes = np.unique(y)
        self.vocab = set()
        
        # Count words for each class
        for text, label in zip(X, y):
            words = self.tokenize(text)
            self.class_counts[label] += 1  # Count of documents in each class
            for word in words:
                self.class_word_counts[label][word] += 1  # Count of word in class
                self.vocab.add(word)  # Add to vocabulary

        # Convert vocab set to list for indexing
        self.vocab = list(self.vocab)

    def predict(self, X):
        predictions = []
        for text in X:
            words = self.tokenize(text)
            class_probs = {}
            for c in self.classes:
                # Prior probability of class
                log_prior = np.log(self.class_counts[c] / sum(self.class_counts.values()))
                # Likelihood of text given class
                log_likelihood = 0
                for word in words:
                    word_count = self.class_word_counts[c][word]
                    total_words = sum(self.class_word_counts[c].values())
                    log_likelihood += np.log((word_count + self.alpha) / (total_words + self.alpha * len(self.vocab)))
                # Combine prior and likelihood
                class_probs[c] = log_prior + log_likelihood

            # Choose class with highest probability
            predictions.append(max(class_probs, key=class_probs.get))
        return predictions


In [56]:
model = NaiveBayes()

In [57]:
# Example training data
X_train = [
    "Get rich quick with our new scheme",  # Spam
    "Meeting at 2 PM today",               # Not Spam
    "Win a free iPhone now",               # Spam
    "Project report due tomorrow",         # Not Spam
    "Claim your lottery prize",            # Spam
    "Lunch at the cafeteria at 12",        # Not Spam
]

y_train = ['spam', 'not spam', 'spam', 'not spam', 'spam', 'not spam']


In [58]:
model.fit(X_train, y_train)

In [64]:
model.predict(X_test)

[np.str_('spam'), np.str_('not spam')]